# HW10：RFM 與分群

This homework is written in the newer Polars and hvPlot library.

In [1]:
import polars as pl

In [2]:
df = pl.read_csv("tafeng5000.csv")
df

,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE
i64,str,i64,str,str,i64,i64,i64,i64,i64
0,"""1/7/2001""",235419,"""30-34""","""115""",110105,4710198221113,1,25,29
1,"""2/13/2001""",1550900,"""35-39""","""115""",590312,4710735405006,1,103,119
2,"""2/26/2001""",909228,""">65""","""115""",110411,4710088414120,2,48,66
3,"""11/18/2000""",1380231,"""30-34""","""221""",130401,4710057870391,1,71,81
4,"""12/1/2000""",1695595,"""45-49""","""221""",530110,4710186161209,2,152,198
…,…,…,…,…,…,…,…,…,…
4995,"""12/26/2000""",1573879,"""55-59""","""221""",500806,26333005345,1,97,132
4996,"""2/3/2001""",2090818,"""35-39""","""221""",130205,4717332008883,1,75,95
4997,"""11/15/2000""",198127,"""30-34""","""115""",760229,733879271232,1,209,299


# Data cleanup

進行描述性統計。

In [3]:
df.describe()

statistic,,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE
str,f64,str,f64,str,str,f64,f64,f64,f64,f64
"""count""",5000.0,"""5000""",5000.0,"""4851""","""5000""",5000.0,5000.0,5000.0,5000.0,5000.0
"""null_count""",0.0,"""0""",0.0,"""149""","""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",2499.5,null,1.4048e6,null,null,285985.2382,4.4571e12,1.3528,102.4436,120.803
"""std""",1443.520003,null,806133.233696,null,null,227810.005443,1.6876e12,1.043528,202.563897,227.340672
"""min""",0.0,"""1/1/2001""",4947.0,"""25-29""","""105""",100101.0,2.0009199e7,1.0,3.0,4.0
"""25%""",1250.0,null,960229.0,null,null,110106.0,4.7101e12,1.0,35.0,42.0
"""50%""",2500.0,null,1.590937e6,null,null,130102.0,4.7104e12,1.0,62.0,75.0
"""75%""",3749.0,null,1.848335e6,null,null,520437.0,4.7127e12,1.0,110.0,129.0
"""max""",4999.0,"""2/9/2001""",2.0002e7,""">65""","""Unknown""",780510.0,9.5576e12,30.0,9428.0,9980.0


發現 `AGE_GROUP` 有 149 個缺失值。可能代表這行資料有問題，故將其移除。

In [4]:
df = df.drop_nulls()

df.describe()

statistic,,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE
str,f64,str,f64,str,str,f64,f64,f64,f64,f64
"""count""",4851.0,"""4851""",4851.0,"""4851""","""4851""",4851.0,4851.0,4851.0,4851.0,4851.0
"""null_count""",0.0,"""0""",0.0,"""0""","""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",2500.127396,null,1.3844e6,null,null,286803.855081,4.4562e12,1.350649,102.737168,121.181612
"""std""",1444.597999,null,614551.822521,null,null,228134.802251,1.6889e12,1.036702,204.853477,229.755808
"""min""",0.0,"""1/1/2001""",7795.0,"""25-29""","""105""",100101.0,2.0009199e7,1.0,3.0,4.0
"""25%""",1246.0,null,951319.0,null,null,110106.0,4.7101e12,1.0,35.0,42.0
"""50%""",2502.0,null,1.584318e6,null,null,130102.0,4.7104e12,1.0,61.0,75.0
"""75%""",3753.0,null,1.849332e6,null,null,520437.0,4.7128e12,1.0,110.0,129.0
"""max""",4999.0,"""2/9/2001""",2.179605e6,""">65""","""Unknown""",780510.0,9.5576e12,30.0,9428.0,9980.0


發現到 `TRANSACTION_DT` 是 `str` 而非 `datetime`，故進行轉換。

In [5]:
df = df.with_columns(pl.col("TRANSACTION_DT").str.strptime(pl.Date, "%m/%d/%Y"))

df.describe()

statistic,,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE
str,f64,str,f64,str,str,f64,f64,f64,f64,f64
"""count""",4851.0,"""4851""",4851.0,"""4851""","""4851""",4851.0,4851.0,4851.0,4851.0,4851.0
"""null_count""",0.0,"""0""",0.0,"""0""","""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",2500.127396,"""2000-12-30 09:48:20.927000""",1.3844e6,null,null,286803.855081,4.4562e12,1.350649,102.737168,121.181612
"""std""",1444.597999,null,614551.822521,null,null,228134.802251,1.6889e12,1.036702,204.853477,229.755808
"""min""",0.0,"""2000-11-01""",7795.0,"""25-29""","""105""",100101.0,2.0009199e7,1.0,3.0,4.0
"""25%""",1246.0,"""2000-11-27""",951319.0,null,null,110106.0,4.7101e12,1.0,35.0,42.0
"""50%""",2502.0,"""2001-01-01""",1.584318e6,null,null,130102.0,4.7104e12,1.0,61.0,75.0
"""75%""",3753.0,"""2001-01-30""",1.849332e6,null,null,520437.0,4.7128e12,1.0,110.0,129.0
"""max""",4999.0,"""2001-02-28""",2.179605e6,""">65""","""Unknown""",780510.0,9.5576e12,30.0,9428.0,9980.0


計算總銷量。

In [6]:
df = df.with_columns(SALES_FIGURE=pl.col("AMOUNT") * pl.col("SALES_PRICE"))

df

,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE,SALES_FIGURE
i64,date,i64,str,str,i64,i64,i64,i64,i64,i64
0,2001-01-07,235419,"""30-34""","""115""",110105,4710198221113,1,25,29,29
1,2001-02-13,1550900,"""35-39""","""115""",590312,4710735405006,1,103,119,119
2,2001-02-26,909228,""">65""","""115""",110411,4710088414120,2,48,66,132
3,2000-11-18,1380231,"""30-34""","""221""",130401,4710057870391,1,71,81,81
4,2000-12-01,1695595,"""45-49""","""221""",530110,4710186161209,2,152,198,396
…,…,…,…,…,…,…,…,…,…,…
4995,2000-12-26,1573879,"""55-59""","""221""",500806,26333005345,1,97,132,132
4996,2001-02-03,2090818,"""35-39""","""221""",130205,4717332008883,1,75,95,95
4997,2000-11-15,198127,"""30-34""","""115""",760229,733879271232,1,209,299,299


# Question 1：計算 RFM

請計算 `tafeng5000` 的 RFM，$R$ 的計算終止日以 2001/12/31 日為標的。

In [7]:
from datetime import datetime

today = datetime(2001, 12, 31)

In [8]:
rfm_df = df.group_by(pl.col("CUSTOMER_ID").alias("CustomerID")).agg(
    monetary=pl.col("SALES_FIGURE").sum(),
    recency=(today - pl.col("TRANSACTION_DT").max()).dt.total_days(),
    frequency=pl.col("TRANSACTION_DT").n_unique(),
)

rfm_df.sort("CustomerID")

CustomerID,monetary,recency,frequency
i64,i64,i64,u32
7795,263,397,1
13697,79,385,1
18173,85,314,1
19897,222,376,1
20220,78,413,2
…,…,…,…
2177267,49,316,1
2178448,115,310,1
2179230,29,307,1


In [9]:
rfm_df.describe()

statistic,CustomerID,monetary,recency,frequency
str,f64,f64,f64,f64
"""count""",4075.0,4075.0,4075.0,4075.0
"""null_count""",0.0,0.0,0.0,0.0
"""mean""",1.3977e6,250.270429,362.64319,1.148466
"""std""",609323.391768,815.242039,35.929324,0.472924
"""min""",7795.0,4.0,306.0,1.0
"""25%""",980302.0,52.0,331.0,1.0
"""50%""",1.594164e6,99.0,357.0,1.0
"""75%""",1.861204e6,199.0,396.0,1.0
"""max""",2.179605e6,27000.0,425.0,10.0


frequency 的資料有點奇特，故先畫直方圖看看數值分佈情況。

In [10]:
rfm_df["frequency"].plot.hist()

alt.Chart(...)

# Question 2：五等分分群

手動分群。

- 將 RFM 各切成 5 等份
- 等級最高給 5 分，最低給 1 分
- 請問 `555` 最有價值群
    1. RFM 平均數各為多少？
    2. 此群佔總人數多少百分比？

根據上面 quantile，將 RFM 分群。

In [11]:
rfm_df = rfm_df.with_columns(
    # Recency 分數（越小越好）
    recency_score=pl.col("recency").qcut(5, labels=["5", "4", "3", "2", "1"]),
    # Frequency 分數（越大越好）
    # FIXME: qcut allow duplicates
    frequency_score=pl.col("frequency").qcut(
        5, labels=["1", "2", "3", "4", "5"], allow_duplicates=True
    ),
    # Monetary 分數（越大越好）
    monetary_score=pl.col("monetary").qcut(5, labels=["1", "2", "3", "4", "5"]),
)

rfm_df

CustomerID,monetary,recency,frequency,recency_score,frequency_score,monetary_score
i64,i64,i64,u32,cat,cat,cat
1672848,65,379,1,"""2""","""1""","""2"""
1969085,145,424,1,"""1""","""1""","""4"""
1980226,65,369,1,"""3""","""1""","""2"""
2060187,10,319,1,"""5""","""1""","""1"""
909174,162,386,1,"""2""","""1""","""4"""
…,…,…,…,…,…,…
591935,288,421,1,"""1""","""1""","""5"""
191746,28,394,1,"""2""","""1""","""1"""
1617122,399,336,1,"""4""","""1""","""5"""


寫出最有價值群 `customer_score`。

In [12]:
rfm_df = rfm_df.with_columns(
    customer_score=pl.concat_str(
        pl.col("recency_score"), pl.col("frequency_score"), pl.col("monetary_score")
    )
)

rfm_df

CustomerID,monetary,recency,frequency,recency_score,frequency_score,monetary_score,customer_score
i64,i64,i64,u32,cat,cat,cat,str
1672848,65,379,1,"""2""","""1""","""2""","""212"""
1969085,145,424,1,"""1""","""1""","""4""","""114"""
1980226,65,369,1,"""3""","""1""","""2""","""312"""
2060187,10,319,1,"""5""","""1""","""1""","""511"""
909174,162,386,1,"""2""","""1""","""4""","""214"""
…,…,…,…,…,…,…,…
591935,288,421,1,"""1""","""1""","""5""","""115"""
191746,28,394,1,"""2""","""1""","""1""","""211"""
1617122,399,336,1,"""4""","""1""","""5""","""415"""


篩選出 `555` 最有價值群。

In [13]:
best_customers = rfm_df.filter(pl.col("customer_score") == "555")

best_customers

CustomerID,monetary,recency,frequency,recency_score,frequency_score,monetary_score,customer_score
i64,i64,i64,u32,cat,cat,cat,str
540711,355,313,2,"""5""","""5""","""5""","""555"""
1241853,1677,315,2,"""5""","""5""","""5""","""555"""
462891,318,319,2,"""5""","""5""","""5""","""555"""
1058369,398,306,3,"""5""","""5""","""5""","""555"""
1555028,704,315,2,"""5""","""5""","""5""","""555"""
…,…,…,…,…,…,…,…
1524963,826,314,4,"""5""","""5""","""5""","""555"""
296106,682,310,2,"""5""","""5""","""5""","""555"""
2058603,371,318,3,"""5""","""5""","""5""","""555"""


最有價值群的 RFM 平均數。

In [14]:
best_customers.select(
    pl.col("monetary").mean(), pl.col("recency").mean(), pl.col("frequency").mean()
)

monetary,recency,frequency
f64,f64,f64
964.305882,314.223529,2.670588


此群佔總人數多少百分比？

In [15]:
print("Best customers: {:.2f}%".format(len(best_customers) / len(rfm_df) * 100))

Best customers: 2.09%


# Question 3：兩等份分群

手動分群：將 F、R 各切成 2 等份（高與低），然後依 F、R 高低分為四群，並給予命名：

- **常貴客**
- **一次客**
- **新顧客**
- **流失客**

In [16]:
rfm_df = rfm_df.with_columns(
    recency_level=pl.col("recency").qcut(2, labels=["Low", "High"]),
    frequency_level=pl.col("frequency").cut([1], labels=["Low", "High"]),
)

rfm_df

CustomerID,monetary,recency,frequency,recency_score,frequency_score,monetary_score,customer_score,recency_level,frequency_level
i64,i64,i64,u32,cat,cat,cat,str,cat,cat
1672848,65,379,1,"""2""","""1""","""2""","""212""","""High""","""Low"""
1969085,145,424,1,"""1""","""1""","""4""","""114""","""High""","""Low"""
1980226,65,369,1,"""3""","""1""","""2""","""312""","""High""","""Low"""
2060187,10,319,1,"""5""","""1""","""1""","""511""","""Low""","""Low"""
909174,162,386,1,"""2""","""1""","""4""","""214""","""High""","""Low"""
…,…,…,…,…,…,…,…,…,…
591935,288,421,1,"""1""","""1""","""5""","""115""","""High""","""Low"""
191746,28,394,1,"""2""","""1""","""1""","""211""","""High""","""Low"""
1617122,399,336,1,"""4""","""1""","""5""","""415""","""Low""","""Low"""


In [17]:
rfm_df = rfm_df.with_columns(
    pl.when((pl.col("recency_level") == "Low") & (pl.col("frequency_level") == "Low"))
    .then(pl.lit("流失客"))
    .when((pl.col("recency_level") == "Low") & (pl.col("frequency_level") == "High"))
    .then(pl.lit("新顧客"))
    .when((pl.col("recency_level") == "High") & (pl.col("frequency_level") == "Low"))
    .then(pl.lit("先前客"))
    .when((pl.col("recency_level") == "High") & (pl.col("frequency_level") == "High"))
    .then(pl.lit("常貴客"))
    .otherwise(pl.lit(None))  # Optional: handle unexpected cases
    .alias("customer_level")
)

rfm_df

CustomerID,monetary,recency,frequency,recency_score,frequency_score,monetary_score,customer_score,recency_level,frequency_level,customer_level
i64,i64,i64,u32,cat,cat,cat,str,cat,cat,str
1672848,65,379,1,"""2""","""1""","""2""","""212""","""High""","""Low""","""先前客"""
1969085,145,424,1,"""1""","""1""","""4""","""114""","""High""","""Low""","""先前客"""
1980226,65,369,1,"""3""","""1""","""2""","""312""","""High""","""Low""","""先前客"""
2060187,10,319,1,"""5""","""1""","""1""","""511""","""Low""","""Low""","""流失客"""
909174,162,386,1,"""2""","""1""","""4""","""214""","""High""","""Low""","""先前客"""
…,…,…,…,…,…,…,…,…,…,…
591935,288,421,1,"""1""","""1""","""5""","""115""","""High""","""Low""","""先前客"""
191746,28,394,1,"""2""","""1""","""1""","""211""","""High""","""Low""","""先前客"""
1617122,399,336,1,"""4""","""1""","""5""","""415""","""Low""","""Low""","""流失客"""


In [18]:
rfm_df["customer_level"].value_counts()

customer_level,count
str,u32
"""新顧客""",359
"""常貴客""",124
"""先前客""",1902
"""流失客""",1690


# Question 4：計算主力產品

承上，針對常貴客群進行分析，計算此群前五項主力商品。

In [19]:
rfm_df.filter(pl.col("customer_level") == "常貴客").join(
    df, left_on="CustomerID", right_on="CUSTOMER_ID"
)["PRODUCT_ID"].value_counts().sort("count", descending=True).head(5)

PRODUCT_ID,count
i64,u32
4711271000014,6
4714981010038,3
4710134023276,2
4710126200067,2
4710011401128,2


# Question 5：$K$-means 分群

將 RFM 以 $K$-means 分群，分群後，畫出各群分布圖（$F$ 值為 X 軸，$R$ 值為 Y 軸），並標示出中心點。

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

## K-means 分群

進行標準化。

In [21]:
scale = StandardScaler()

# 只取 RFM 欄位
rfm_df_v = rfm_df.select(["recency", "frequency", "monetary"])

normalized_rfm_df = pl.from_numpy(
    scale.fit_transform(rfm_df_v.to_numpy()), rfm_df_v.columns
)

normalized_rfm_df

recency,frequency,monetary
f64,f64,f64
0.455305,-0.313971,-0.227286
1.707918,-0.313971,-0.129144
0.176947,-0.313971,-0.227286
-1.214845,-0.313971,-0.294759
0.650156,-0.313971,-0.108288
…,…,…
1.62441,-0.313971,0.046286
0.872843,-0.313971,-0.272677
-0.741635,-0.313971,0.182458


使用 Silhouette 分數找出最適合分群數量。

In [22]:
for n in range(3, 12):
    kmeans = KMeans(n_clusters=n, random_state=486).fit(normalized_rfm_df.to_pandas())
    silhouette_avg = silhouette_score(
        normalized_rfm_df.to_pandas(), kmeans.labels_, random_state=512
    )

    print(f"n_clusters: {n}, silhouette_score: {silhouette_avg}")

n_clusters: 3, silhouette_score: 0.5612419443542773
n_clusters: 4, silhouette_score: 0.5713773935321552
n_clusters: 5, silhouette_score: 0.5878957550080073
n_clusters: 6, silhouette_score: 0.5938132165837852
n_clusters: 7, silhouette_score: 0.5231868465851423
n_clusters: 8, silhouette_score: 0.5377529862078199
n_clusters: 9, silhouette_score: 0.5420138688132331
n_clusters: 10, silhouette_score: 0.5513300087974637
n_clusters: 11, silhouette_score: 0.4956188743242425


看起來 $K=5$ ($\mathrm{SS} = 0.58822$) 是最好的分組數量。

> 因為 KMeans 和 Silhouette 分數都有 `random_state`，我沒成功鎖住，所以每次執行結果都不一樣。

In [23]:
kmeans = KMeans(n_clusters=5).fit(normalized_rfm_df.to_pandas())

print(kmeans.labels_)
print(kmeans.cluster_centers_)

[1 1 1 ... 0 1 0]
[[-0.74593233 -0.31397132 -0.06001678]
 [ 1.0346167  -0.31397132 -0.09367144]
 [-0.92919593  4.77153617  0.29558513]
 [-0.54987746  0.97838537 11.20957601]
 [-0.48784647  1.80079417  0.11246003]]


還原標準化後的值，找出各群中心點，可與上面標準化後的值比較。

In [24]:
customer_center_reverse = pl.from_numpy(
    scale.inverse_transform(kmeans.cluster_centers_), rfm_df_v.columns
)
customer_center_reverse.columns = [
    "recency_center",
    "frequency_center",
    "monetary_center",
]

customer_center_reverse

recency_center,frequency_center,monetary_center
f64,f64,f64
335.845634,1.0,201.348233
399.811708,1.0,173.914906
329.261905,3.404762,491.214286
342.888889,1.611111,9387.666667
345.117347,2.0,341.941327


加入各群標籤以及計算各群個數。

In [25]:
four_cluster_df = rfm_df_v.with_columns(pl.Series("cluster", kmeans.labels_))

four_cluster_df

recency,frequency,monetary,cluster
i64,u32,i64,i32
379,1,65,1
424,1,145,1
369,1,65,1
319,1,10,0
386,1,162,1
…,…,…,…
421,1,288,1
394,1,28,1
336,1,399,0


In [26]:
four_cluster_df.group_by("cluster").agg(pl.col("monetary").count()).sort("cluster")

cluster,monetary
i32,u32
0,1924
1,1657
2,84
3,18
4,392


## 各群分佈圖

加入各群標籤以及計算各群個數。

In [27]:
customer_center_reverse.with_row_index()

index,recency_center,frequency_center,monetary_center
u32,f64,f64,f64
0,335.845634,1.0,201.348233
1,399.811708,1.0,173.914906
2,329.261905,3.404762,491.214286
3,342.888889,1.611111,9387.666667
4,345.117347,2.0,341.941327


In [28]:
import altair as alt

(
    customer_center_reverse.with_row_index()
    .plot.scatter(
        x=alt.X("frequency_center", scale=alt.Scale(domain=[0, 10])),
        y="recency_center",
        color=alt.Color("index:N"),
        size=alt.SizeValue(100),
    )
    .properties(title="Cluster Centers")
    + four_cluster_df.with_row_index()
    .plot.scatter(x="frequency", y="recency", color=alt.Color("cluster:N"))
    .properties(title="Clustered Customers")
).configure_scale(zero=False)

alt.LayerChart(...)

# Question 6：泡泡圖

請將各群畫出泡泡圖，X 軸為 frequency，y 軸為 recency,，泡泡大小為每群人數。

In [29]:
four_clusters_cdf = four_cluster_df.group_by("cluster").agg(
    recency=pl.col("recency").mean(),
    frequency=pl.col("frequency").mean(),
    monetary=pl.col("monetary").mean(),
    count=pl.col("recency").count(),
)

four_clusters_cdf.sort("cluster")

cluster,recency,frequency,monetary,count
i32,f64,f64,f64,u32
0,335.845634,1.0,201.348233,1924
1,399.811708,1.0,173.914906,1657
2,329.261905,3.404762,491.214286,84
3,342.888889,1.611111,9387.666667,18
4,345.117347,2.0,341.941327,392


In [30]:
four_clusters_cdf.plot.scatter(
    x=alt.X("frequency"),
    y=alt.Y("recency"),
    color="cluster:N",
    size="count",
).properties(title="各群泡泡圖").configure_scale(zero=False)

alt.Chart(...)

# Question 7：哪一群為最有價值？

print 出此群的 RFM 值。

In [31]:
four_clusters_cdf.sort("cluster")

cluster,recency,frequency,monetary,count
i32,f64,f64,f64,u32
0,335.845634,1.0,201.348233,1924
1,399.811708,1.0,173.914906,1657
2,329.261905,3.404762,491.214286,84
3,342.888889,1.611111,9387.666667,18
4,345.117347,2.0,341.941327,392


由於第二群 recency 為最小，monetary 為第二高，frequency 為最高，故此群為最有價值。

In [34]:
four_cluster_df.filter(pl.col("cluster") == 2).select(
    recency=pl.col("recency").mean(),
    frequency=pl.col("frequency").mean(),
    monetary=pl.col("monetary").sum(),
)

recency,frequency,monetary
f64,f64,i64
329.261905,3.404762,41262


# Question 8：推薦產品

針對最有價值群那一群顧客，列出每個人的推薦商品，請依顧客編號排名，print前5筆結果。

In [35]:
(
    four_cluster_df.with_columns(rfm_df["CustomerID"])
    .sort("CustomerID")
    .filter(pl.col("cluster") == 1)
    .join(df, left_on="CustomerID", right_on="CUSTOMER_ID")
    .select(
        pl.col("CustomerID").alias("customer_id"),
        pl.col("PRODUCT_ID").alias("product_id"),
        pl.col("SALES_FIGURE").alias("sales_figure"),
    )
    .group_by("customer_id")
    .agg(
        product_id=pl.col("product_id")
        .filter(pl.col("sales_figure") == pl.col("sales_figure").max())
        .sort()
        .first(),
    )
    .sort("customer_id")
    .head(5)
)

customer_id,product_id
i64,i64
7795,4710498234202
13697,4710054139422
19897,4710985010272
21821,4901638025938
23474,4710175567128
